In [3]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.utils import load_img, img_to_array
from time import time
from matplotlib import pyplot as plt
import os

In [94]:
image = load_img('C:/Users/Eliza Marie/Desktop/thesis 2/training/test/tamban/IMG20230303195404 256 x 256.jpg', 
                 target_size=(224, 224))

In [95]:
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print(image.shape)

(1, 224, 224, 3)


In [38]:
from keras.utils import normalize
image = normalize(image, axis=1)

In [41]:
###############################################
###PREDICT USING REGULAR KERAS TRAINED MODEL FILE (h5). 
##########################################################
keras_model_size = os.path.getsize("new_Fish_v27/")/1048576  #Convert to MB
print("Keras Model size is: ", keras_model_size, "MB")
#Using regular keral model
model = tf.keras.models.load_model("new_Fish_v27/")

Keras Model size is:  0.00390625 MB


In [45]:
time_before=time()
keras_prediction = model.predict(image)
time_after=time()
total_keras_time = time_after - time_before
print("Total prediction time for keras model is: ", total_keras_time)

1/1 [==============================] - 7s 7s/step
Total prediction time for keras model is:  7.935150861740112


In [46]:
print("The keras prediction for this image is: ", keras_prediction, " 0=tamban, 1=turingan")

The keras prediction for this image is:  [[0.07952833 0.92047167]]  0=tamban, 1=turingan


In [55]:
##################################################################################
#### PREDICT USING tflite ###
############################################################################
tflite_size = os.path.getsize("unoptim_Fish_v27.tflite")/1048576  #Convert to MB
print("tflite Model without opt. size is: ", tflite_size, "MB")
#Not optimized (file size = 540MB). Taking about 0.5 seconds for inference
tflite_model_path = "unoptim_Fish_v27.tflite"

tflite Model without opt. size is:  97.6204605102539 MB


In [56]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [96]:
# Test the model on input data.
input_shape = input_details[0]['shape']
print(input_shape)

# Load image
input_data = image

input_data = np.expand_dims(input_data, axis=0)

[1 1 1 3]


In [98]:
# Test the model on input data.
input_shape = input_details[0]['shape']
print(input_shape)

import random
# Load image
#input_data1 = image
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

#


[1 1 1 3]


In [99]:
#test_image = np.expand_dims(test_image, axis=0)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [69]:
time_before=time()
interpreter.invoke()
time_after=time()
total_tflite_time = time_after - time_before
print("Total prediction time for tflite without opt model is: ", total_tflite_time)

output_data_tflite = interpreter.get_tensor(output_details[0]['index'])
print("The tflite w/ opt prediction for this image is: ", output_data_tflite, " 0=tamban, 1=turingan")

Total prediction time for tflite without opt model is:  0.0
The tflite w/ opt prediction for this image is:  [[0.31520438 0.6847956 ]]  0=tamban, 1=turingan


In [76]:
#################################################################
#### PREDICT USING tflite with optimization###
#################################################################
tflite_optimized_size = os.path.getsize("Fish_v27.tflite")/1048576  #Convert to MB
print("tflite Model with optimization size is: ", tflite_optimized_size, "MB")
#Optimized using default optimization strategy (file size = 135MB). Taking about 39 seconds for prediction
tflite_optimized_model_path = "Fish_v27.tflite"

tflite Model with optimization size is:  48.84135055541992 MB


In [77]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_optimized_model_path)
interpreter.allocate_tensors()

In [78]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [79]:
# Test the model on input data.
input_shape = input_details[0]['shape']
print(input_shape)

[1 1 1 3]


In [88]:
import random
# Load image
#input_data1 = image
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

In [89]:
interpreter.set_tensor(input_details[0]['index'], input_data)

In [91]:
time_before=time()
interpreter.invoke()
time_after=time()
total_tflite_opt_time = time_after - time_before
print("Total prediction time for tflite model with opt is: ", total_tflite_opt_time)



Total prediction time for tflite model with opt is:  0.015027761459350586


In [92]:
output_data_tflite_opt = interpreter.get_tensor(output_details[0]['index'])
print("The tflite with opt prediction for this image is: ", output_data_tflite_opt, " 0=Uninfected, 1=Parasited")

The tflite with opt prediction for this image is:  [[0.3153619  0.68463814]]  0=Uninfected, 1=Parasited


In [93]:
#Summary
print("###############################################")
print("Keras Model size is: ", keras_model_size)
print("tflite Model without opt. size is: ", tflite_size)
print("tflite Model with optimization size is: ", tflite_optimized_size)
print("________________________________________________")
print("Total prediction time for keras model is: ", total_keras_time)
print("Total prediction time for tflite without opt model is: ", total_tflite_time)
print("Total prediction time for tflite model with opt is: ", total_tflite_opt_time)
print("________________________________________________")
print("The keras prediction for this image is: ", keras_prediction, " 0=Uninfected, 1=Parasited")
print("The tflite w/o opt prediction for this image is: ", output_data_tflite, " 0=Uninfected, 1=Parasited")
print("The tflite with opt prediction for this image is: ", output_data_tflite_opt, " 0=Uninfected, 1=Parasited")

###############################################
Keras Model size is:  0.00390625
tflite Model without opt. size is:  97.6204605102539
tflite Model with optimization size is:  48.84135055541992
________________________________________________
Total prediction time for keras model is:  7.935150861740112
Total prediction time for tflite without opt model is:  0.0
Total prediction time for tflite model with opt is:  0.015027761459350586
________________________________________________
The keras prediction for this image is:  [[0.07952833 0.92047167]]  0=Uninfected, 1=Parasited
The tflite w/o opt prediction for this image is:  [[0.31520438 0.6847956 ]]  0=Uninfected, 1=Parasited
The tflite with opt prediction for this image is:  [[0.3153619  0.68463814]]  0=Uninfected, 1=Parasited
